In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import functools
import itertools

In [73]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 0#np.array([0, 0, 0, 1])
    elif i % 5  == 0: return 1 #np.array([0, 0, 1, 0])
    elif i % 3  == 0: return 2#np.array([0, 1, 0, 0])
    else:             return 3#np.array([1, 0, 0, 0])

def invert(label, sample):
    if label == 3:
        return sample
    else:
        return {0:'fizzbuzz', 1:'fizz',2:'buzz'}[label]


In [78]:
def generate_training(num_digits, therange):
    for i in range(*therange):
        yield binary_encode(i, num_digits)
names=[str(i) for i in range(12)]
train_x = pd.DataFrame((i for i in generate_training(12,(101,1024))), columns=names)
test_range = (0,100)
train_range = (101,1024)
train_y = np.asarray([fizz_buzz_encode(i) for i in range(*train_range)])
test_x = pd.DataFrame((i for i in generate_training(12,test_range)),columns=names)
test_y = np.asarray([fizz_buzz_encode(i) for i in range(*test_range)])
def gen(x,y):
    for idx, row in x.iterrows():
        data = np.asarray(row)
        label = y[idx]
        yield data, label
def train_input_fn(features, labels, batch_size=20, repeat=100):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    #dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))#.map(namer)
    dataset = tf.data.Dataset.from_generator(lambda: gen(features, labels)
                                             ,output_types=(tf.int64, tf.int64)
                                             ,output_shapes = (tf.TensorShape([12]),
                                                               tf.TensorShape([]))
                                            )
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat(repeat).batch(batch_size)
    # Return the dataset.
    iterator = dataset.make_one_shot_iterator()
    d,l = iterator.get_next()
    return {'image':d}, l
def test_input_gen(features, labels, batch_size=20):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    #dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))#.map(namer)
    dataset = tf.data.Dataset.from_generator(lambda: gen(features, labels)
                                             ,output_types=(tf.int64, tf.int64)
                                             ,output_shapes = (tf.TensorShape([12]),
                                                               tf.TensorShape([]))
                                            )
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.batch(batch_size)
    # Return the dataset.
    iterator = dataset.make_one_shot_iterator()
    d,l = iterator.get_next()
    return {'image':d}, l
def slice_input_fn(features, labels, batch_size = 20, repeat=100):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat(repeat).batch(batch_size)
    # Return the dataset.
    iterator = dataset.make_one_shot_iterator()
    d,l = iterator.get_next()
    return d,l
def test_input_fn(features, labels, batch_size=20):
    """An input function for evaluation or prediction"""
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))#.map(namer)
    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    # Return the dataset.
    iterator = dataset.make_one_shot_iterator()
    data, labels = iterator.get_next()
    return data, labels

In [79]:
def main(train_x, train_y, test_x, test_y):
    old_feature_columns = [tf.feature_column.numeric_column(key=str(k))
                         for k in range(12)]
    my_feature_columns = [tf.feature_column.numeric_column('image',shape=12)]
    our_input_fn = test_input_fn
    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # The model must choose between 4 classes
        n_classes=4,
        # Two hidden layers of 100 nodes each.
        hidden_units=[100, 100]
        )

    # Train the Model.
    classifier.train(input_fn=lambda:train_input_fn(train_x, train_y))

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda:test_input_gen(test_x, test_y))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
    predictions = classifier.predict(
        input_fn=lambda:test_input_gen(test_x,test_y))
    formatted = [(invert(pred.get('class_ids')[0], idx),idx)
                for idx, pred in enumerate(predictions)]
    expected = [(invert(fizz_buzz_encode(i),i),i) for i, p in enumerate(test_y)]
    return expected, formatted
e, f = main(train_x, train_y, test_x, test_y)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f759d6ef198>, '_log_step_count_steps': 100, '_task_type': 'worker', '_tf_random_seed': None, '_master': '', '_save_summary_steps': 100, '_task_id': 0, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_service': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_model_dir': '/tmp/tmpjf_kcswr'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpjf_kcswr/model.ckpt.
INFO:tensorflow:loss = 29.334274, step = 1
INFO:tensorflow:global_step/sec: 116.837
INFO:tensorflow:loss = 23.863882, step = 101 (0.858 sec)
INFO:tensorflow:global_step/sec: 123.528
INFO:tensorflow:loss = 22.351671, step = 201 (0.809 sec)
INFO:tensorflow:global_step/sec: 122.035
INFO:tensorflow:loss =

In [80]:
for expected, got in zip(e,f):
    print('answer is {}, neural net predicted {} {}'.format(expected, got,('wrong' if expected!=got else '')))

answer is ('fizzbuzz', 0), neural net predicted ('fizzbuzz', 0) 
answer is (1, 1), neural net predicted (1, 1) 
answer is (2, 2), neural net predicted (2, 2) 
answer is ('buzz', 3), neural net predicted ('buzz', 3) 
answer is (4, 4), neural net predicted (4, 4) 
answer is ('fizz', 5), neural net predicted ('fizz', 5) 
answer is ('buzz', 6), neural net predicted ('buzz', 6) 
answer is (7, 7), neural net predicted (7, 7) 
answer is (8, 8), neural net predicted (8, 8) 
answer is ('buzz', 9), neural net predicted ('buzz', 9) 
answer is ('fizz', 10), neural net predicted ('fizz', 10) 
answer is (11, 11), neural net predicted (11, 11) 
answer is ('buzz', 12), neural net predicted ('buzz', 12) 
answer is (13, 13), neural net predicted (13, 13) 
answer is (14, 14), neural net predicted (14, 14) 
answer is ('fizzbuzz', 15), neural net predicted ('fizzbuzz', 15) 
answer is (16, 16), neural net predicted (16, 16) 
answer is (17, 17), neural net predicted (17, 17) 
answer is ('buzz', 18), neural n

In [50]:
f

[('even', 0),
 ('odd', 1),
 ('odd', 2),
 ('even', 3),
 ('even', 4),
 ('even', 5),
 ('odd', 6),
 ('even', 7),
 ('odd', 8),
 ('odd', 9),
 ('odd', 10),
 ('even', 11),
 ('odd', 12),
 ('odd', 13),
 ('odd', 14),
 ('even', 15),
 ('even', 16),
 ('even', 17),
 ('odd', 18),
 ('even', 19),
 ('even', 20),
 ('even', 21),
 ('odd', 22),
 ('even', 23),
 ('odd', 24),
 ('even', 25),
 ('odd', 26),
 ('even', 27),
 ('odd', 28),
 ('odd', 29),
 ('even', 30),
 ('even', 31),
 ('even', 32),
 ('even', 33),
 ('odd', 34),
 ('even', 35),
 ('even', 36),
 ('odd', 37),
 ('even', 38),
 ('even', 39),
 ('odd', 40),
 ('odd', 41),
 ('even', 42),
 ('odd', 43),
 ('odd', 44),
 ('even', 45),
 ('odd', 46),
 ('odd', 47),
 ('even', 48),
 ('odd', 49),
 ('even', 50),
 ('odd', 51),
 ('even', 52),
 ('even', 53),
 ('even', 54),
 ('odd', 55),
 ('odd', 56),
 ('odd', 57),
 ('odd', 58),
 ('even', 59),
 ('odd', 60),
 ('odd', 61),
 ('even', 62),
 ('even', 63),
 ('even', 64),
 ('odd', 65),
 ('odd', 66),
 ('even', 67),
 ('odd', 68),
 ('even',